# Simulation for Prescriptive Analytics

In this notebook, we will discuss how to use simulation for prescriptive analytics - in other words, how to use discrete event simulations to help make decisions. 

In order to demonstrate a decsion-making scenario, we will return to the repair facility simulation from an earlier lesson from this week. We slightly modify our simulation function – we now make both the number of spare machines `S_0` and the failure rate `lambda` input variables.

In [1]:
import math 
import random
random.seed(10)

def run_repair_facility_sim(n_max,S_0=5,failrate=1.0):
  #initialize variables as empty lists
  A = [0]*n_max #arrival times
  D = [0]*n_max #departure times
  F = [0]*n_max #inter-failure times
  S = [0]*n_max #repair times
  #run simulation
  for n in range(n_max):
    F[n]=-math.log(random.random())/failrate
    #use rejection method to simulate repair time
    ok=False
    while ok==False:
      x=random.random(); y=random.random()
      if x<0.6:
        if y<(5/3)*x:
          ok=True
      else:
        if y<2.5-2.5*x:
          ok=True
    #compute variables      
    S[n]=x
    A[n]=A[n-1]+F[n]
    D[n]=S[n]+max(A[n],D[n-1])
    #stopping condition
    if n>S_0 and A[n]<D[n-S_0]:
      
      break
    
  return A, D, F, S, n

Suppose we are managing the factory described in this simulation. We think that by increasing the average time until the factory halts production, we can improve profits. There are a few ways we might increase this average time. One is to increase the number of spare machines – if the number of spare machines increases, then we will have more backups before a failure brings the floor below full production. Another option is to reduce the failure rate of machines, perhaps by spending more resources on maintenance. Both of these options have costs associated with them: purchasing more machines and paying specialists to spend time maintaining the machines.

Let average time to a halt in production be $H$, and suppose we estimate that our revenue is related to $H$ as $100 \log(H)$  - the logarithm suggests diminishing returns to increasing time to halt. Furthermore, suppose we quantify the costs of maintenance as $9000(1-\lambda)^4$ - such that our current maintenance costs are zero but reducing failure rate becomes more and more costly. Also, each new machine beyond what we have costs 175 – in other words, machine cost is $175(S_0-5)$. We can then compute profit as revenue minus costs. What combination of investment in new machines and maintenance will maximize our profit? Or are the costs of investment simply too great compared to the improvement in halting time they will result in?

This code below runs through 3 different failure rates and 3 possible numbers of spare machines, and records costs, average time to halting, and profit. Each simulation configuration is run 100 times, over which an average is taken. `n_max` is set to a very high number, so that simulations do not run out of machine failures before halting. Outcomes are printed in a `DataFrame` table.

In [2]:
import pandas as pd
import numpy as np

#run simulation with different failure rates and number of spare machines
#NOTE: simulations take approximately 1 minute to run

#set possible decisions
failure_rates = [1.0,0.8,0.6]
possible_spare_machines = [5,6,7]

rows = [] #initialize rows for recording each experiment
for failrate in failure_rates:
  for S_0 in possible_spare_machines:
    #record time to factory failure
    Halting_Times = []
    for sim in range(100): #run each sim experiment 100 times
        n_max= 1000000 # high n_nax  
        A, D, F, S, n = run_repair_facility_sim(n_max,S_0=S_0,failrate=failrate)
        Halting_Times.append(A[n])
        #compute cost
    cost = 9000*(1-failrate)**4 + 175*(S_0-5)
    #calculate averge time to failure for this experiment
    avg_time_to_halt = np.mean(Halting_Times)
    #compute profit for this experiment
    profit = 100*np.log(avg_time_to_halt)-cost    
    #record experiment outcomes
    rows.append([failrate,S_0,cost,avg_time_to_halt,profit])
    print("simulating with lambda = %s, S_0 = %s" % (failrate,S_0))

#create data table with outcomes
pd.DataFrame(rows,columns=["lambda","S_0","cost", " avg. time to halt","profit"])

simulating with lambda = 1.0, S_0 = 5
simulating with lambda = 1.0, S_0 = 6
simulating with lambda = 1.0, S_0 = 7
simulating with lambda = 0.8, S_0 = 5
simulating with lambda = 0.8, S_0 = 6
simulating with lambda = 0.8, S_0 = 7
simulating with lambda = 0.6, S_0 = 5
simulating with lambda = 0.6, S_0 = 6
simulating with lambda = 0.6, S_0 = 7


,lambda,S_0,cost,avg. time to halt,profit
0,1.0,5,0.0,352.170800,586.411629
1,1.0,6,175.0,811.897212,494.937375
2,1.0,7,350.0,2353.576448,426.369134
3,0.8,5,14.4,1300.787356,702.672502
4,0.8,6,189.4,4334.980564,648.047241
5,0.8,7,364.4,19501.588193,623.425119
6,0.6,5,230.4,7514.620436,662.060579
7,0.6,6,405.4,45292.081735,666.688750
8,0.6,7,580.4,280725.339973,674.113203


The results indicate that the profit maximizing option is a failure rate $\lambda$ of 0.8, and an `S_0` of 5. In other words, we have determined by simulation that our best option out of the possibilities tested is to invest in a small amount of maintenance, but purchase no more machines.